In [89]:
import nltk
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC as SupportVectorClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Concatenate, Embedding, Input, Bidirectional, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow_addons.activations import mish
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import sys
import re
import numpy as np
import pandas as pd
import random
import contractions

from collections import deque, Counter
from itertools import permutations, repeat, combinations




In [90]:
empires = []
vectorizer = CountVectorizer()
min_sentence_length = 15 # average length of a sentence is between 15 and 20 words
level_sample_sizes = True

In [91]:
with open("Resources/Data/EmpireText.txt", "r", encoding="utf8") as file:
    line = file.readline().replace("\n", "").split(" ")
    for empire in line:
        empires.append(empire)

In [92]:
def format_sentence(sentence, current_empire):
        if len(sentence.split(" ")) > min_sentence_length:
            sentence = contractions.fix(sentence)
            contains_other_empire = False
            for empire in empires:
                if empire == current_empire:
                    continue
                elif re.search(empire.lower(), sentence.lower()) is not None:
                    contains_other_empire = True

            if not contains_other_empire:
                return sentence.strip()

text = []
labels = []
with open("Resources/Data/EmpireText.txt", "r", encoding="utf8") as file: 
    for line in file.readlines()[1:]:
        line = line.strip().replace("\n", "")
        line = re.sub("\[.{0,4}]", "", line) #remove wikipedia citings
        line = line.replace("C.", "C")
        line = line.replace("E.", "E")
        line = line.replace("D.", "D")
        
        if line in empires:
            current_empire = line
            continue
        elif len(line.replace(" ", "")) < min_sentence_length:
            continue
        elif len(line.split(" ")) < min_sentence_length:
            continue
        elif line[-1] != ".":
            line += "."
    
        line = nltk.sent_tokenize(line)
        formatted_line = ""
        for sentence in line:
            sentence = format_sentence(sentence, current_empire)
            if sentence is not None:
                formatted_line += sentence + " "
                text.append(sentence)
                labels.append(current_empire)
            
data = pd.DataFrame({"label":labels, "text":text})

# Data Visualization

## Training Data

In [93]:
data.head()

,label,text
0,Roman,"The Roman Empire, the ancient empire, centred ..."
1,Spanish,The Spanish Empire (Spanish: Imperio españold)...
2,Russian,"The Russian Empire, also known as Imperial Rus..."
3,Roman,A period of unrest and civil wars in the 1st c...
4,Roman,This period encompassed the career of Julius C...


In [94]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540 entries, 0 to 2539
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   2540 non-null   object
 1   text    2540 non-null   object
dtypes: object(2)
memory usage: 39.8+ KB


In [95]:
training_value_counts = data["label"].value_counts()
training_value_counts

Roman      989
Russian    800
Spanish    751
Name: label, dtype: int64

# Text classification:
### The process of text classification is comprised of 4 main steps
#### - Preprocessing the text
#### - Encoding labels
#### - Vectorizing the text
#### - Training the model(s)
***

## Text preprocessing
### Steps:
- Lowercasing
- Tokenization
- POS tagging
- Lemmatization

Note: When tested, the models showed better results without the removal of stopwords

### Example:

In [96]:
example_paragraph = data['text'][1]
example_paragraph = "".join(example_paragraph)
print(data['label'][1], "empire:")
example_paragraph

Spanish empire:


'The Spanish Empire (Spanish: Imperio españold), also known as the Hispanic Monarchy (Spanish: Monarquía Hispánica) or the Catholic Monarchy (Spanish: Monarquía Católica) was a colonial empire governed by Spain and its predecessor states between 1492 and 1976.'

In [97]:
lowercase_paragraph = example_paragraph.lower()
lowercase_paragraph

'the spanish empire (spanish: imperio españold), also known as the hispanic monarchy (spanish: monarquía hispánica) or the catholic monarchy (spanish: monarquía católica) was a colonial empire governed by spain and its predecessor states between 1492 and 1976.'

In [98]:
tokenized_paragraph = nltk.word_tokenize(lowercase_paragraph)
for word in tokenized_paragraph:
    print(f"'{word}'", end=", ")

'the', 'spanish', 'empire', '(', 'spanish', ':', 'imperio', 'españold', ')', ',', 'also', 'known', 'as', 'the', 'hispanic', 'monarchy', '(', 'spanish', ':', 'monarquía', 'hispánica', ')', 'or', 'the', 'catholic', 'monarchy', '(', 'spanish', ':', 'monarquía', 'católica', ')', 'was', 'a', 'colonial', 'empire', 'governed', 'by', 'spain', 'and', 'its', 'predecessor', 'states', 'between', '1492', 'and', '1976', '.', 

In [99]:
tagged_paragraph = nltk.pos_tag(tokenized_paragraph)
for word in tagged_paragraph:
    print(word, end=', ')

('the', 'DT'), ('spanish', 'JJ'), ('empire', 'NN'), ('(', '('), ('spanish', 'JJ'), (':', ':'), ('imperio', 'NN'), ('españold', 'NN'), (')', ')'), (',', ','), ('also', 'RB'), ('known', 'VBN'), ('as', 'IN'), ('the', 'DT'), ('hispanic', 'JJ'), ('monarchy', 'NN'), ('(', '('), ('spanish', 'JJ'), (':', ':'), ('monarquía', 'NN'), ('hispánica', 'NN'), (')', ')'), ('or', 'CC'), ('the', 'DT'), ('catholic', 'JJ'), ('monarchy', 'NN'), ('(', '('), ('spanish', 'JJ'), (':', ':'), ('monarquía', 'NN'), ('católica', 'NN'), (')', ')'), ('was', 'VBD'), ('a', 'DT'), ('colonial', 'JJ'), ('empire', 'NN'), ('governed', 'VBN'), ('by', 'IN'), ('spain', 'NN'), ('and', 'CC'), ('its', 'PRP$'), ('predecessor', 'NN'), ('states', 'NNS'), ('between', 'IN'), ('1492', 'CD'), ('and', 'CC'), ('1976', 'CD'), ('.', '.'), 

In [100]:
def get_pos(tag):    
    if tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN
        
wnl = WordNetLemmatizer()
lemmatized_sentence = [wnl.lemmatize(word, get_pos(pos)) for word, pos in tagged_paragraph]
for word in lemmatized_sentence:
    print(f"'{word}'", end=", ")

'the', 'spanish', 'empire', '(', 'spanish', ':', 'imperio', 'españold', ')', ',', 'also', 'know', 'a', 'the', 'hispanic', 'monarchy', '(', 'spanish', ':', 'monarquía', 'hispánica', ')', 'or', 'the', 'catholic', 'monarchy', '(', 'spanish', ':', 'monarquía', 'católica', ')', 'be', 'a', 'colonial', 'empire', 'govern', 'by', 'spain', 'and', 'it', 'predecessor', 'state', 'between', '1492', 'and', '1976', '.', 

### Application

In [101]:
sentences = []
labels = []
wnl = WordNetLemmatizer()
for row in data.iterrows():
    label = row[1]['label']
    text = row[1]['text']
    text = nltk.word_tokenize(text)
    text = nltk.pos_tag(text)
    text = [wnl.lemmatize(word, get_pos(pos)) for word, pos in text]
    sentences.append(text)
    labels.append(label)

preprocessed_data = pd.DataFrame({"label":labels, "text":sentences})

In [102]:
preprocessed_data.head()

,label,text
0,Roman,"[The, Roman, Empire, ,, the, ancient, empire, ..."
1,Spanish,"[The, Spanish, Empire, (, Spanish, :, Imperio,..."
2,Russian,"[The, Russian, Empire, ,, also, know, a, Imper..."
3,Roman,"[A, period, of, unrest, and, civil, war, in, t..."
4,Roman,"[This, period, encompass, the, career, of, Jul..."


## Label Encoding

In [103]:
labels = []
for label in preprocessed_data['label']:
    labels.append(empires.index(label))
preprocessed_data['label'] = labels

In [104]:
preprocessed_data.head()

,label,text
0,0,"[The, Roman, Empire, ,, the, ancient, empire, ..."
1,1,"[The, Spanish, Empire, (, Spanish, :, Imperio,..."
2,2,"[The, Russian, Empire, ,, also, know, a, Imper..."
3,0,"[A, period, of, unrest, and, civil, war, in, t..."
4,0,"[This, period, encompass, the, career, of, Jul..."


## Text Vectorization

In [105]:
vectorizer = CountVectorizer()
text = []
for sentence in preprocessed_data['text']:
    text.append(" ".join(sentence))
vectorizer.fit(text)
vectorized_text = vectorizer.transform(text)

## Training the models

In [106]:
uniques = set()
for sentence in data['text']:
    for word in sentence:
        uniques.add(word)
num_uniques = len(uniques)

In [107]:
text = vectorized_text
labels = preprocessed_data['label']
x_train, x_test, y_train, y_test = train_test_split(text, labels, train_size=0.9)

#### Decision Tree

In [108]:
decision_tree = DecisionTreeClassifier(random_state=2, max_depth=150, max_features=1000)
decision_tree.fit(x_train, y_train)
decision_tree_score = decision_tree.score(x_test, y_test)
decision_tree_score

0.8070866141732284

#### Random Forest

In [109]:
random_forest = RandomForestClassifier(random_state=0, n_estimators=300, max_depth=150, max_features=1000)
random_forest.fit(x_train, y_train)
random_forest_score = random_forest.score(x_test, y_test)
random_forest_score

0.8937007874015748

#### Support Vector Machine

In [110]:
best_c = 0.1
best_kernel = "linear"
support_vector_machine = SupportVectorClassifier(kernel=best_kernel, C=best_c)
support_vector_machine.fit(x_train, y_train)
support_vector_machine_score = support_vector_machine.score(x_test, y_test)
support_vector_machine_score

0.9291338582677166

### Long Short Term Memory

Sidenote: while one could convert x_Train from SparseTensor to array it causes the lstm model to have low accuracy and take 20 min per epoch, so its better to just format the data with tensorflow methods to convert it to a format that tensorflow is better equipped to handle.

In [111]:
text = preprocessed_data['text']
labels = preprocessed_data['label']
longest_sentence = max(text, key=lambda x: len(x))
encoded_sentences = [one_hot(" ".join(sentence), num_uniques) for sentence in text]
padded_sequences = pad_sequences(encoded_sentences, maxlen=len(longest_sentence), padding='post')

def encode_labels(labels):
    output = []
    for label in labels:
        label_array = np.zeros(len(empires))
        label_array[label] = 1    
        output.append(label_array)
    return output

labels = encode_labels(labels)
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, labels, train_size=.9, random_state=463)
print("input example:", x_train[0])
print("label example:", y_train[0])

input example: [ 14  46  50 101  14  81  14  36  28  42  43  14  40 115  50  96  58  21
 104  23  32   4  17  22  50  81 109  28  81  78  14  62  72  21 103  52
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0]
label example: [1. 0. 0.]


In [128]:
lstm = Sequential()
lstm.add(Embedding(num_uniques, 32))
lstm.add(Bidirectional(LSTM(64)))
lstm.add(Dropout(0.2))
lstm.add(Dense(256, activation="relu"))

lstm.add(Dense(3, activation="sigmoid"))
opt = Adam(learning_rate=0.0001)
lstm.compile(optimizer=opt, loss='categorical_crossentropy', metrics=["accuracy"])

In [129]:
mc = ModelCheckpoint('best-weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)
lstm.fit(np.array(x_train), np.array(y_train), epochs=100, verbose=1, batch_size=64, callbacks=[mc], validation_data=(x_test, np.array(y_test)))

Epoch 1/100
36/36 [==============================] - 5s 74ms/step - loss: 1.0962 - accuracy: 0.3784 - val_loss: 1.0923 - val_accuracy: 0.4094
Epoch 2/100
36/36 [==============================] - 2s 61ms/step - loss: 1.0936 - accuracy: 0.3871 - val_loss: 1.0880 - val_accuracy: 0.4094
Epoch 3/100
36/36 [==============================] - 2s 61ms/step - loss: 1.0910 - accuracy: 0.3871 - val_loss: 1.0840 - val_accuracy: 0.4094
Epoch 4/100
36/36 [==============================] - 2s 62ms/step - loss: 1.0884 - accuracy: 0.3871 - val_loss: 1.0806 - val_accuracy: 0.4094
Epoch 5/100
36/36 [==============================] - 2s 62ms/step - loss: 1.0829 - accuracy: 0.3871 - val_loss: 1.0673 - val_accuracy: 0.4094
Epoch 6/100
36/36 [==============================] - 2s 63ms/step - loss: 1.0637 - accuracy: 0.4274 - val_loss: 1.0312 - val_accuracy: 0.5236
Epoch 7/100
36/36 [==============================] - 2s 63ms/step - loss: 1.0361 - accuracy: 0.4689 - val_loss: 1.0228 - val_accuracy: 0.5433
Epoch 

In [ ]:
lstm_score = lstm.evaluate(x_test, np.array(y_test), verbose=1)
lstm_acc = lstm_score[1]*100
print(f'Test accuracy: {lstm_acc}')

3/3 [==============================] - 1s 36ms/step - loss: 10.0673 - accuracy: 0.5571
Test accuracy: 55.714285373687744
